In [1]:
import polars as pl
import pyodbc
from datetime import date

In [2]:
cierre = "202504"
cierre_l = "Abr25"
cierre_d = date(2025,4,30)

# Carpetas
wd = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = wd + "data/raw/"
wd_data_external = wd + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/dwh/"
wd_data_processed_fotos = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/fotos/"

# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_db_dwh_nr = wd_data_processed_fotos + f'parquet/DB_DWH_NR_{cierre}.parquet'
fl_db_dwh_r = wd_data_processed_fotos + f'parquet/DB_DWH_R_{cierre}.parquet'
fl_prem = wd_data_raw + f'Prem_{cierre}.mdb'

fl_recuperadas_detalle = wd_data_processed_dwh + f'parquet/Recupera_con_Pagos_Flujos_{cierre}.parquet'
fl_pagadas_detalle = wd_data_processed_dwh + f'parquet/Pagadas_Global_VF_{cierre}.parquet'

# Outputs


### Macro Saldos

In [19]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)

estados = pl.read_database(query='SELECT * FROM ESTADOS', connection=conn, schema_overrides={'Estado_ID': pl.Int32})
sector = pl.read_database(query='SELECT * FROM SECTOR', connection=conn, schema_overrides={'Sector_ID': pl.Int32})
estrato = pl.read_database(query='SELECT * FROM ESTRATO', connection=conn, schema_overrides={'Estrato_Id': pl.Int32})
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
tipo_garantia = pl.read_database(query='SELECT * FROM [TIPO_GARANTIA]', connection=conn, schema_overrides={'Tipo_garantia_ID': pl.Int32})
sfc = pl.read_database(query='SELECT * FROM [SIN FONDOS CONTRAGARANTIA]', connection=conn)

conn.close()

In [27]:
db_dwh_nr = pl.read_parquet(fl_db_dwh_nr)
db_dwh_r = pl.read_parquet(fl_db_dwh_r)

In [28]:
def garantias_c_saldo(df):
    """ Filtra garantías con saldo positivo.  """
    result = df.filter(pl.col("Saldo_Contingente_Mn") > 0)
    return result

db_dwh_nr = garantias_c_saldo(db_dwh_nr)
db_dwh_r = garantias_c_saldo(db_dwh_r)

In [29]:
def cruza_catalogos(df):
    """Incorpora catálogos. """
    # sfc_aux = sfc.select(
    #     pl.col("Intermediario_Id"),
    #     pl.col("CLAVE_CREDITO"),
    #     pl.col("FONDOS_CONTRAGARANTIA").alias("CSF")
    # )

    result = (df.join(estados, left_on="ESTADO_ID", right_on="Estado_ID", how="left")
              .join(estrato, left_on="ESTRATO_ID", right_on="Estrato_Id", how="left")
              .join(sector, left_on="SECTOR_ID", right_on="Sector_ID", how="left")
              .join(tipo_garantia.select(['Tipo_garantia_ID', 'Tipo_garantia']), left_on='TIPO_GARANTIA_ID', right_on='Tipo_garantia_ID', how='left')
              .join(tipo_credito.select(['Tipo_Credito_ID', 'Tipo_Credito']), left_on='TIPO_CREDITO_ID', right_on="Tipo_Credito_ID", how='left')
              #.join(sfc_aux, left_on=['INTERMEDIARIO_ID', 'NUMERO_CREDITO'], right_on=['Intermediario_Id', 'CLAVE_CREDITO'], how='left')
              )
    
    result = result.with_columns(pl.col("CSF").fill_null("CF"))
    return result

db_dwh_r = cruza_catalogos(db_dwh_r)
db_dwh_nr = cruza_catalogos(db_dwh_nr)

In [34]:
def agrupa_bd_con_saldo(df):
    result = df.group_by([
        "BUCKET", 
        "Producto", 
        "TIPO_CREDITO_ID", 
        "MM_UDIS", 
        "TPRO_CLAVE", 
        "NR_R", 
        "CSG", 
        "INTERMEDIARIO_ID", 
        "Razón Social (Intermediario)", 
        "Tipo_Persona", 
        "Porcentaje de Comisión Garantia", 
        "Porcentaje Garantizado", 
        "AGRUPAMIENTO", 
        "Estado", 
        "Sector", 
        "Estrato", 
        "Tipo_garantia", 
        "Tipo_Credito", 
        "PROGRAMA_ID",  
        "CSF"
    ]).agg(
        pl.col("Saldo_Contingente_Mn").sum().alias("SALDO_MN"),
        pl.col("NUMERO_CREDITO").count().alias("TOT_CREDITOS")
    )
    return result

db_dwh_r_vf = agrupa_bd_con_saldo(db_dwh_r)
db_dwh_nr_vf = agrupa_bd_con_saldo(db_dwh_nr)

In [36]:
base_agrup_final = pl.concat([db_dwh_r_vf, db_dwh_nr_vf], rechunk=True)

In [37]:
base_agrup_final

BUCKET,Producto,TIPO_CREDITO_ID,MM_UDIS,TPRO_CLAVE,NR_R,CSG,INTERMEDIARIO_ID,Razón Social (Intermediario),Tipo_Persona,Porcentaje de Comisión Garantia,Porcentaje Garantizado,AGRUPAMIENTO,Estado,Sector,Estrato,Tipo_garantia,Tipo_Credito,PROGRAMA_ID,CSF,SALDO_MN,TOT_CREDITOS
i16,str,i32,i32,i32,str,str,str,str,str,f32,f32,str,str,str,str,str,str,i32,str,f64,u32
3,"""GARANTIA EMPRESARIAL""",3,0,310168,"""R""","""SG""","""10040030""","""BANCO DEL BAJIO""","""F""",3.63,50.0,"""Producto NAFIN""","""ZACATECAS""","""COMERCIO""","""PEQUEÑA EMPRESA""","""SIN GARANTIA""","""CREDITOS EN CUENTA CORRIENTE""",310168,"""SF""",179997.61,2
4,"""GARANTIA EMPRESARIAL""",3,1,310146,"""R""","""SG""","""10000233""","""BANAMEX""","""F""",5.17,50.0,"""Producto NAFIN""","""MEXICO""","""INDUSTRIA""","""PEQUEÑA EMPRESA""","""OBLIGADO SOLIDARIO OPCIONAL""","""CREDITOS EN CUENTA CORRIENTE""",310146,"""SF""",501364.07,2
1,"""GARANTIA EMPRESARIAL""",3,0,310158,"""R""","""SG""","""10040058""","""BANCO REGIONAL DE MONTERREY""","""M""",2.15,50.0,"""Producto NAFIN""","""BAJA CALIFORNIA NORTE""","""COMERCIO""","""MEDIANA EMPRESA""","""OBLIGADO Y/O AVALISTA""","""CREDITOS EN CUENTA CORRIENTE""",310158,"""SF""",1.176816e6,2
5,"""GARANTIA EMPRESARIAL""",3,0,310135,"""R""","""SG""","""10040012""","""BBV BANCOMER""","""M""",5.32,50.0,"""Producto NAFIN""","""SONORA""","""SERVICIOS""","""PEQUEÑA EMPRESA""","""OBLIGADO Y/O AVALISTA""","""CREDITOS EN CUENTA CORRIENTE""",310135,"""SF""",29288.31,2
3,"""GARANTIA EMPRESARIAL""",3,1,310140,"""R""","""SG""","""10040062""","""BANCA AFIRME""","""M""",6.21,50.0,"""Producto NAFIN""","""NUEVO LEON""","""SERVICIOS""","""MEDIANA EMPRESA""","""OBLIGADO Y/O AVALISTA""","""CREDITOS EN CUENTA CORRIENTE""",310140,"""SF""",7.673404e6,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,"""GARANTIA EMPRESARIAL""",2,0,310183,"""NR""","""SG""","""10040030""","""BANCO DEL BAJIO""","""M""",2.5,50.0,"""Producto NAFIN""","""SAN LUIS POTOSI""","""SERVICIOS""","""PEQUEÑA EMPRESA""","""OBLIGADO Y/O AVALISTA""","""CREDITOS SIMPLES""",310183,"""SF""",939241.37,2
5,"""GARANTIA EMPRESARIAL""",2,0,310084,"""NR""","""SG""","""10000171""","""HSBC MEXICO""","""F""",10.51,50.0,"""Producto NAFIN""","""DISTRITO FEDERAL""","""SERVICIOS""","""MICRO EMPRESA""","""OBLIGADO Y/O AVALISTA""","""CREDITOS SIMPLES""",310084,"""SF""",42392.61,2
4,"""GARANTIA EMPRESARIAL""",2,0,310163,"""NR""","""SG""","""10040012""","""BBV BANCOMER""","""F""",4.8,50.0,"""Producto NAFIN""","""HIDALGO""","""SERVICIOS""","""PEQUEÑA EMPRESA""","""OBLIGADO Y/O AVALISTA""","""CREDITOS SIMPLES""",310163,"""SF""",8.3544e6,8


In [ ]:
# VALIDA_SALDO
base_agrup_final.group_by(
    "Producto"
).agg(
    pl.col("SALDO_MN").sum().alias("S_SALDO(MDP)")
)

Producto,S_SALDO(MDP)
str,f64
"""GARANTIA SECTORIAL""",3.5604e10
"""GARANTIA EMPRESARIAL""",1.8162e11
"""GARANTIAS FEFA""",7.5969e8
"""GARANTIA FIANZA""",1.3038e10
"""GARANTIA ESPECIAL""",1.3675e8
"""GARANTIA SUBASTA""",483068.119124
"""EMPRESA MEDIANA""",9.9472e8
"""GARANTIA EMERGENTES""",9.5083e8


#### Prem